In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from keras.models import Sequential
from keras.layers import Dense

# Load data set
df = pd.read_csv('nba_rookie_data.csv')
df.head()


2023-11-02 22:38:12.321567: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-02 22:38:12.380257: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-02 22:38:12.380298: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-02 22:38:12.380331: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-02 22:38:12.389500: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-02 22:38:12.391302: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

,Name,Games Played,Minutes Played,Points Per Game,Field Goals Made,Field Goal Attempts,Field Goal Percent,3 Point Made,3 Point Attempt,3 Point Percent,...,Free Throw Attempts,Free Throw Percent,Offensive Rebounds,Defensive Rebounds,Rebounds,Assists,Steals,Blocks,Turnovers,TARGET_5Yrs
0,Brandon Ingram,36,27.4,7.4,2.6,7.6,34.7,0.5,2.1,25.0,...,2.3,69.9,0.7,3.4,4.1,1.9,0.4,0.4,1.3,0
1,Andrew Harrison,35,26.9,7.2,2.0,6.7,29.6,0.7,2.8,23.5,...,3.4,76.5,0.5,2.0,2.4,3.7,1.1,0.5,1.6,0
2,JaKarr Sampson,74,15.3,5.2,2.0,4.7,42.2,0.4,1.7,24.4,...,1.3,67.0,0.5,1.7,2.2,1.0,0.5,0.3,1.0,0
3,Malik Sealy,58,11.6,5.7,2.3,5.5,42.6,0.1,0.5,22.6,...,1.3,68.9,1.0,0.9,1.9,0.8,0.6,0.1,1.0,1
4,Matt Geiger,48,11.5,4.5,1.6,3.0,52.4,0.0,0.1,0.0,...,1.9,67.4,1.0,1.5,2.5,0.3,0.3,0.4,0.8,1


In [3]:
# Remove outliers using IQR method

# Define a function to calculate the lower and upper bounds for each column
def iqr_bounds(col):
    q1 = col.quantile(0.25)
    q3 = col.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return lower, upper

# Apply the function to each numerical column and store the results in a dictionary
bounds = {}
for col in df.columns[1:-1]: # Exclude the name and target columns
    bounds[col] = iqr_bounds(df[col])

# Remove the rows that have values outside the bounds for any column
df_clean = df.copy()
for col in bounds:
    lower = bounds[col][0]
    upper = bounds[col][1]
    df_clean = df_clean[(df_clean[col] >= lower) & (df_clean[col] <= upper)]

# Check the shape of the cleaned data set
df_clean.shape


(985, 21)

In [5]:
# Split the data set into features and target
X = df_clean.drop(['Name', 'TARGET_5Yrs'], axis=1) # Drop the name and target columns
y = df_clean['TARGET_5Yrs']

# Split the data set into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shapes of the splits
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(788, 19) (197, 19) (788,) (197,)


In [6]:
# Create an instance of LogisticRegression
log_reg = LogisticRegression()

# Fit the model to the training data
log_reg.fit(X_train, y_train)

# Predict the labels for the testing data
y_pred_log_reg = log_reg.predict(X_test)

# Evaluate the performance of the model using accuracy score, confusion matrix, and classification report
acc_log_reg = accuracy_score(y_test, y_pred_log_reg)
cm_log_reg = confusion_matrix(y_test, y_pred_log_reg)
cr_log_reg = classification_report(y_test, y_pred_log_reg)

# Print the results
print('Accuracy score of Logistic Regression:', acc_log_reg)
print('Confusion matrix of Logistic Regression:\n', cm_log_reg)
print('Classification report of Logistic Regression:\n', cr_log_reg)


/home/wanjala/PycharmProjects/7CS030AI/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy score of Logistic Regression: 0.7411167512690355
Confusion matrix of Logistic Regression:
 [[61 28]
 [23 85]]
Classification report of Logistic Regression:
               precision    recall  f1-score   support

           0       0.73      0.69      0.71        89
           1       0.75      0.79      0.77       108

    accuracy                           0.74       197
   macro avg       0.74      0.74      0.74       197
weighted avg       0.74      0.74      0.74       197
